In [1]:
import pandas as pd
from hdfs3 import HDFileSystem
hdfs = HDFileSystem(user='ebouille')
import os
username = os.environ['JUPYTERHUB_USER']
import networkx as nx
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import pickle
import heapq
import pickle
import networkx as nx
import math
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import clear_output

from utils.file_utils import *
from utils.graph_utils import *
from utils.delay_utils import *
from utils.viz_utils import *
from utils.plan_utils import *

In [2]:
!git lfs pull
clear_output()

In [3]:
username = "moiseev" # Use this line if you want to use pretrained models.
# username = os.environ['JUPYTERHUB_USER'] # Uncomment this line if you want to use your models.

In [4]:
print("Loading models...")
stops = read_parquet_from_hdfs("stops", username)
trips = read_parquet_from_hdfs("trips", username)
routes = read_parquet_from_hdfs("routes", username)
trip_to_product = trip_to_product_func(username)

direc = "/work/final-assignment-group-y/data/"
filename = "graph_reverse.pickle"
with open(direc + filename, 'rb') as config_dictionary_file:
    G_rev = pickle.load(config_dictionary_file)
    
scaler = read_scaler_from_hdfs("delays_scaler_model", username)
log = read_logistic_regression_from_hdfs("delays_log_model", username)

gt = GraphTraversals(None, G_rev)
clear_output()

### Choose journey details

In [5]:
style = {'description_width': '200px'}
layout = widgets.Layout(width='auto', height='auto')

In [6]:
stop_options = list(zip(stops.stop_name, stops.stop_id))
stop_options.sort(key = lambda x: x[0])
origin_dropdown = widgets.Combobox(
    value=stop_options[0][0]+': '+str(stop_options[0][1]),
    placeholder='Choose Origin',
    options=[i[0]+': '+str(i[1]) for i in stop_options],
    description='Origin:',
    ensure_option=True,
    disabled=False,
    style=style,
    layout=layout,
)

In [7]:
destination_dropdown = widgets.Combobox(
    value=stop_options[1][0]+': '+str(stop_options[1][1]),
    placeholder='Choose Destination',
    options=[i[0]+': '+str(i[1]) for i in stop_options],
    description='Destination:',
    ensure_option=True,
    disabled=False,
    style=style,
    layout=layout, 
)

In [8]:
time_widget = widgets.Text(
    value="14:00",
    description='Pick arrival time',
    disabled=False,
    style=style,
    layout=layout,
)

In [9]:
date_widget = widgets.Text(
    value="14.05.2019",
    description='Pick travel date',
    disabled=False,
    style=style,
    layout=layout,
)

In [10]:
confidence_threshold = widgets.BoundedFloatText(
    value=0.8,
    min=0,
    max=1.0,
    step=0.01,
    description='Confidence threshold',
    disabled=False,
    style=style,
    layout=layout
)

In [11]:
n_routes_widget = widgets.IntText(
    value=1,
    description='Pick number of predicted routes',
    disabled=False,
    style=style,
    layout=layout,
)

In [12]:
def interactive_search(origin, destination, arrival_time, travel_date, n_routes, confidence_threshold):
    clear_output()
    print("Computation in progress...")
    origin = int(origin.split(': ')[1])
    destination = int(destination.split(': ')[1])
    A = plan_travel(gt, origin, destination, arrival_time, travel_date, 
                    n_routes, confidence_threshold, fast=True, 
                    scaler=scaler, model=log, product_id_map=product_id_map, trip_to_product=trip_to_product)
    fig = visualize(A, trips, routes, stops)
    clear_output()
    fig.show()

In [13]:
widgets.interactive(
    interactive_search, 
    origin=origin_dropdown,
    destination=destination_dropdown,
    arrival_time=time_widget,
    travel_date=date_widget,
    n_routes=n_routes_widget,
    confidence_threshold=confidence_threshold)

interactive(children=(Combobox(value='Adlikon b. R., Dorf: 508', description='Origin:', ensure_option=True, la…